
# To use this notebook:
* This notebook was designed to run in a UNIX like environment (Linux, Mac, WSL)
* Enable any required Bedrock models in the target AWS account.
* From a terminal, activate a conda or other Python virtual environment where dependencies can be installed.
* Be sure that the same Python/conda/virtual environment is selected as a kernel for the notebook before executing.
* Run `make install` from the `idp_common_pkg/` directory to install the `idp_common` Python library and its dependencies.
* Copy the `.env.example` file in the notebooks folder to a file named `.env`
* Customize the `IDP_INPUT_BUCKET_NAME` and `IDP_OUTPUT_BUCKET_NAME`, as well as any other values like `AWS_PROFILE` and `MLFLOW_TRACKING_URI` to their proper values.
* Copy any files referenced in the notebook to the proper location in the input/output S3 buckets.
___

In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.display import Markdown
from dotenv import load_dotenv
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
import os, sys
load_dotenv()

In [ ]:
# Define the S3 locations for input and output
input_bucket_name =  os.getenv("IDP_INPUT_BUCKET_NAME")
output_bucket_name = os.getenv("IDP_OUTPUT_BUCKET_NAME")
filename = "insurance_package.pdf"
input_object_key = f"input-docs/samples/{filename}"

input_s3_uri = f"s3://{input_bucket_name}/{input_object_key}"
output_s3_uri = f"s3://{input_bucket_name}/output-docs"
blueprintArn=os.getenv("BDA_BLUEPRINT_ARN")

In [ ]:
from idp_common.bda.bda_service import BdaService


bda_service = BdaService(
    output_s3_uri=output_s3_uri
)

In [ ]:
result = bda_service.invoke_data_automation(input_s3_uri=input_s3_uri, blueprintArn=blueprintArn)

In [ ]:
import json

display(Markdown(f"``` json\n{json.dumps(result, indent=2)}\n```"))

In [ ]:
from dataclasses import asdict

from idp_common.bda.bda_invocation import BdaInvocation

bda_invocation = BdaInvocation.from_s3(s3_url=result["output_location"])

In [ ]:
display(Markdown(f"``` json\n{json.dumps(asdict(bda_invocation), indent=2)}\n```"))

In [ ]:
custom_output = bda_invocation.get_custom_output()

display(Markdown(f"``` json\n{json.dumps(custom_output, indent=2)}\n```"))